# Let's Read Ancient Papers with Cold Coffee: The Simplest Way

## The Ancient Coffee Mystery ☕🏺

**Picture this:** You're a barista-archaeologist staring at a coffee scroll (papyrus) that's been carbonized into volcanic ash toast for 2,000 years. 
☠️ It's so fragile that touching it = instant coffee dust. 

But inside? Priceless Roman coffee recipes!

**Your Mission:** Use CT scans (X-ray vision for coffee!) to find the papyrus layers without physically unrolling. You're mapping the coffee surface through folds, gaps, and ash damage, like finding a single coffee bean in a crumpled napkin! 🎯

**Why This is HARD :**

Super fragile - One wrong move = ☠️

Burnt to a crisp - Looks like ash, hard to distinguish

Crazy folded - Like origami from hell

Topology matters - Can't glue layers together or split them apart!

Noise everywhere - Ash creates fake coffee artifacts

## 1: ☕  ☕ Let's Gather Our Coffee Ingredients (Imports & Setup)
Load all necessary libraries and define global parameters used throughout the pipeline

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from tqdm.auto import tqdm
from PIL import Image, ImageSequence
from io import BytesIO
import zipfile
from scipy import ndimage


# Our coffee recipe parameters

TOP_PERCENT = 25.0          # How much coffee foam to keep (top 25% brightest)
MIN_COMPONENT_VOXELS = 400  # Remove tiny coffee grounds (noise)
CLOSING_ITERS = 1           # Gentle stirring (gap filling)

## 2: 🏗️ Building Our Espresso Machine (Neural Network)
Every great cold coffee needs a powerful espresso machine! This is our 3D U-Net - it's like a super smart coffee filter that knows exactly which beans (voxels) are the good stuff :

In [2]:
class DoubleConv3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 4, padding=2),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, 4, padding=2),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class UNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, init_features=24):
        super().__init__()
        f = init_features
        self.enc1 = DoubleConv3D(in_channels, f)
        self.pool1 = nn.MaxPool3d(2)
        self.enc2 = DoubleConv3D(f, f*2)
        self.pool2 = nn.MaxPool3d(2)
        self.enc3 = DoubleConv3D(f*2, f*4)
        self.pool3 = nn.MaxPool3d(2)
        self.bottleneck = DoubleConv3D(f*4, f*8)
        self.upconv3 = nn.ConvTranspose3d(f*8, f*4, 2, stride=2)
        self.dec3 = DoubleConv3D(f*8, f*4)
        self.upconv2 = nn.ConvTranspose3d(f*4, f*2, 2, stride=2)
        self.dec2 = DoubleConv3D(f*4, f*2)
        self.upconv1 = nn.ConvTranspose3d(f*2, f, 2, stride=2)
        self.dec1 = DoubleConv3D(f*2, f)
        self.out = nn.Conv3d(f, out_channels, 1)
    
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))
        b = self.bottleneck(self.pool3(e3))
        d3 = self.upconv3(b)
        d3 = torch.cat([d3, e3], 1)
        d3 = self.dec3(d3)
        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, e2], 1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, e1], 1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.out(d1))

def load_tiff(path):
    with Image.open(path) as tif:
        return np.stack([np.array(frame) for frame in ImageSequence.Iterator(tif)])

def save_to_zip(volume, zip_file, filename):
    volume = (np.clip(volume, 0, 1) * 255).astype(np.uint8)
    pages = [Image.fromarray(v) for v in volume]
    buffer = BytesIO()
    pages[0].save(buffer, format='TIFF', save_all=True, append_images=pages[1:])
    zip_file.writestr(filename, buffer.getvalue())


##  3: ⚡ Our Coffee Grinding & Brewing Tools (Core Functions)

Time to grind those coffee beans! These functions are our coffee grinder, blender, and strainer, turning raw CT scans into perfect predictions!

In [3]:
def dl_inference_fast(volume, model, device, stride=72):
    vol = (volume - volume.mean()) / (volume.std() + 1e-8)
    Z,H,W = vol.shape
    ps = 96
    out = np.zeros((Z,H,W), dtype=np.float32)
    cnt = np.zeros((Z,H,W), dtype=np.float32)
    
    with torch.no_grad():
        for z in range(0, max(1,Z-ps+1), stride):
            for y in range(0, max(1,H-ps+1), stride):
                for x in range(0, max(1,W-ps+1), stride):
                    ze,ye,xe = min(z+ps,Z), min(y+ps,H), min(x+ps,W)
                    patch = vol[z:ze,y:ye,x:xe]
                    if patch.shape != (ps,ps,ps):
                        pad = np.zeros((ps,ps,ps))
                        pad[:patch.shape[0],:patch.shape[1],:patch.shape[2]] = patch
                        patch = pad
                    pred = model(torch.from_numpy(patch[None,None]).float().to(device))[0,0].cpu().numpy()
                    out[z:ze,y:ye,x:xe] += pred[:ze-z,:ye-y,:xe-x]
                    cnt[z:ze,y:ye,x:xe] += 1
    
    return out / np.maximum(cnt, 1)

def classical_cv_slice_adaptive(volume):
    filtered = ndimage.median_filter(volume, size=(1, 3, 3))
    masks = []
    percentile_cutoff = 100.0 - TOP_PERCENT
    for sl in filtered:
        t = np.percentile(sl, percentile_cutoff)
        masks.append((sl > t).astype(np.float32))
    return np.stack(masks)

def slice_adaptive_ensemble(classical, dl):
    ensemble = np.zeros_like(classical)
    for i in range(classical.shape[0]):
        slice_classical = classical[i]
        slice_dl = dl[i]
        weight = 0.7 + 0.2 * np.abs(slice_classical - 0.5)
        ensemble[i] = weight * slice_classical + (1 - weight) * slice_dl
    return ensemble

def topology_aware_clean(mask):
    cc_structure = ndimage.generate_binary_structure(rank=3, connectivity=1)
    labeled, num = ndimage.label(mask, structure=cc_structure)
    
    if num == 0:
        return mask.astype(np.uint8)
    
    component_sizes = ndimage.sum(mask, labeled, index=np.arange(1, num + 1))
    keep_labels = np.where(component_sizes >= MIN_COMPONENT_VOXELS)[0] + 1
    cleaned = np.isin(labeled, keep_labels).astype(np.uint8)
    
    if CLOSING_ITERS > 0:
        closing_structure = np.zeros((3, 3, 3), dtype=np.uint8)
        closing_structure[1, :, :] = 1
        cleaned = ndimage.binary_closing(cleaned, structure=closing_structure, iterations=CLOSING_ITERS).astype(np.uint8)
    
    return cleaned.astype(np.float32)


##  4: 🚀 Brewing Our Perfect Cold Coffee (Main Pipeline)

Time to brew! Load the model, grab the test data, and run our coffee making factory to create the submission.zip!

In [4]:
print("\n 1️⃣/3️⃣Loading model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#mill jaye to gpu na mille to cpu
model = UNet3D(in_channels=1, out_channels=1, init_features=24).to(device)

model_path = Path('/kaggle/input/model11/best_model_fast.pth')
if model_path.exists():
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Loaded: {model_path}")

model.eval()

print("\n[2️⃣/3️⃣] Now test data")
ROOT = Path('/kaggle/input/vesuvius-challenge-surface-detection')
test_df = pd.read_csv(ROOT / 'test.csv')
test_imgs = ROOT / 'test_images'
print(f"Test volumes: {len(test_df)}")

print("\n[3️⃣/3️⃣] Just 30 sec everthing completed --> Slice-adaptive + DL + Topology")

submission_zip = Path('/kaggle/working/submission.zip')

with zipfile.ZipFile(submission_zip, 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Processing'):
        image_id = row['id']
        img_path = test_imgs / f'{image_id}.tif'
        
        if not img_path.exists():
            continue
        
        try:
            volume = load_tiff(img_path).astype(np.float32)
            
            classical_pred = classical_cv_slice_adaptive(volume)
            
            dl_pred = dl_inference_fast(volume, model, device, stride=72)
            
            ensemble = slice_adaptive_ensemble(classical_pred, dl_pred)
            
            binary_mask = (ensemble > 0.5).astype(np.uint8)
            final = topology_aware_clean(binary_mask)
            
            save_to_zip(final, zf, f'{image_id}.tif')
            
        except Exception as e:
            print(f"ERROR {image_id}: {str(e)[:80]}")

zip_size = submission_zip.stat().st_size / 1e6


print(f"\n{'_-'*60}")
print(f"DONE! submission.zip ({zip_size:.1f} MB)")
print(f"{'_-'*60}")



 1️⃣/3️⃣Loading model

[2️⃣/3️⃣] Now test data
Test volumes: 1

[3️⃣/3️⃣] Just 30 sec everthing completed --> Slice-adaptive + DL + Topology


Processing:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR 1407735: Sizes of tensors must match except in dimension 1. Expected size 30 but got size

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
DONE! submission.zip (0.0 MB)
_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
